##### 네이버 맞춤법 API 활용(막상 맞춤법 틀린부분(을,를 등)을 잘 잡아내지 못하고 선수 이름이나 '샅바'같은 멀쩡한 단어들을 오히려 이상하게 수정해놓는 문제 발생, 그래서 일단은 맞춤법 코드는 사용하지 않았음)

In [4]:
# conda activate 가상환경, cd py-hanspell-master3, python setup.py install
from hanspell import spell_checker

In [5]:
import re
def split_sentences(text):
    sentences = re.split(r'(?<=[.?!])\s', text)
    result = []
    current_sentence = ''
    
    for sentence in sentences:
        if len(current_sentence) + len(sentence) + 1 <= 500:  # +1 for space after sentence
            current_sentence += sentence + ' '
        else:
            if current_sentence:
                result.append(current_sentence.strip())
            current_sentence = sentence + ' '
    
    if current_sentence:
        result.append(current_sentence.strip()) 
    
    return result


def spell_check_over500(text):
    total_modified_sentence = ''
    total_errors = 0
    total_modified_words = []

    
    sentence_list = split_sentences(text)
    for sentence in sentence_list:
        modified_dic = spell_checker.check(sentence)
        total_modified_sentence += modified_dic.as_dict()['checked']
        total_errors += modified_dic.as_dict()['errors']
        modified_words = [key for key, value in modified_dic.as_dict()['words'].items() if value != 0]
        total_modified_words += modified_words
    
    return {'modified_sentence':  total_modified_sentence, 'modified_words': total_modified_words, 'errors': total_errors}


#### 크롤링한 데이터를 이용하여 기사작성

In [6]:
import pandas as pd
import numpy as np
import json
import re

In [7]:
df1 = pd.read_csv('C:/Users/qudwn/OneDrive/바탕 화면/와이빅타_여름방학프로젝트/2차프로젝트/빅콘테스트_비정형데이터/금강장사경기_final.csv', index_col=0)

In [8]:
df2 = pd.read_csv('C:/Users/qudwn/OneDrive/바탕 화면/와이빅타_여름방학프로젝트/2차프로젝트/빅콘테스트_비정형데이터/금강장사경기_문자중계.csv', index_col=0)

In [9]:
# 함수에 대입할 최종 데이터프레임
df_ = pd.concat([df2[['contest_name', 'play_date', 'event', 'play_category', 'winner', 'match_result']], df1], axis=1)
df_.head()

,contest_name,play_date,event,play_category,winner,match_result,play_level,match_result,text_relay,player_name_A,...,weight_A,height_A,weight_B,height_B,specialized_technique_A,specialized_technique_B,first_learning_year_A,first_learning_year_B,pro_debut_year_A,pro_debut_year_B
0,보은장사씨름대회,5.20(토),개인전,금강,승리자 청 정재욱,0승 0패 (기본),1조 32강,승리자 청 정재욱,경기종료 \n청 승\n,정재욱,...,82kg,175cm,95kg,176cm,안다리,들배지기,1997년,2004년,2010년,2017년
1,보은장사씨름대회,5.20(토),개인전,금강,승리자 홍 김기선,0승 0패 (기본),1조 32강,승리자 홍 김기선,경기종료 \n홍 승,홍성용,...,90kg,178cm,92kg,172cm,오금당기기,오금당기기,2004년,2002년,2015년,2015년
2,보은장사씨름대회,5.20(토),개인전,금강,승리자 홍 김민정,2승 1패,1조 32강,승리자 홍 김민정,1라운드 경기시작\n58초\n홍 김민정\n홍 김민정 허리기술(들배지기) 공격시도\n...,이설빈,...,85kg,178cm,93kg,187cm,왼배지기,들배지기,2007년,2000년,2020년,2011년
3,보은장사씨름대회,5.20(토),개인전,금강,승리자 승 자 없음,-승 -패,1조 32강,승리자 승 자 없음,경기종료 동시 기권\n승자 없음,윤대호,...,90kg,185cm,95kg,186cm,들배지기,밭다리,2000년,2008년,2005년,2019년
4,보은장사씨름대회,5.20(토),개인전,금강,승리자 홍 이민호,0승 0패 (기본),1조 32강,승리자 홍 이민호,경기종료 \n홍 승,황정훈,...,90kg,183cm,93kg,180cm,안다리,들배지기,2008년,2003년,2021년,2017년


In [10]:
import re

def merge_sentences(text):  # 문자중계에 나와있는 단어들을 문장 형태로 조합하는 함수, 밑에 코드에서 다른 함수를 정의할 때 사용
    pattern = r"(\d+)초가 경과되었을 때 ([\w\s\(\)]+) 공격시도\. (\d+)초가 경과되었을 때 ([\w\s\(\)]+) 기술방어\."
    
    def change(match):
        start_time1 = match.group(1)
        action1 = match.group(2)
        start_time2 = match.group(3)
        action2 = match.group(4)
        merged_sentence = f" {start_time1}초가 경과되었을 때 {action1} 공격시도를 하였고 이를 {action2} 기술방어하였다."
        return merged_sentence
    
    modified_text = re.sub(pattern, change, text)
    modified_text = re.sub(r'(\d+초가 경과되었을 때 [^0-9]+) 승리기술\.', r'\1을 성공하여 해당 라운드를 승리하였다.', modified_text)
    modified_text = modified_text.replace('합의판정', '합의판정을 받았다')
    modified_text = modified_text.replace('  ', ' ')
    modified_text = modified_text.replace('. ', '.\n')

    return modified_text

original_text = '2초가 경과되었을 때 홍샅바 김민정 선수가 허리기술(들배지기) 공격시도. 2초가 경과되었을 때 청샅바 이설빈 선수가 기술방어. 3초가 경과되었을 때 홍샅바 김민정 선수가 다리기술(호미걸이) 공격시도. 3초가 경과되었을 때 청샅바 이설빈 선수가 기술방어. 5초가 경과되었을 때 청샅바 이설빈 선수가 다리기술(안다리걸기) 승리기술.'

modified_text = merge_sentences(original_text)

print(modified_text)


 2초가 경과되었을 때 홍샅바 김민정 선수가 허리기술(들배지기) 공격시도를 하였고 이를 청샅바 이설빈 선수가 기술방어하였다.
3초가 경과되었을 때 홍샅바 김민정 선수가 다리기술(호미걸이) 공격시도를 하였고 이를 청샅바 이설빈 선수가 기술방어하였다.
5초가 경과되었을 때 청샅바 이설빈 선수가 다리기술(안다리걸기)을 성공하여 해당 라운드를 승리하였다.


In [11]:
def text_relay_modifier(df): # 위에서 편집한 데이터프레임을 받아 각 row마다 정보들을 조합하여 기사를 생성하는 함수
    for index, row in df.iterrows():

        print('\n{} row processing...'.format(index+1))
        
        contest_name = row['contest_name']
        play_date = row['play_date']
        event = row['event']
        play_category = row['play_category']
        winner = row['winner']
        play_level = row['play_level']
        match_result = row['match_result']
        text_relay = row['text_relay']
        player_name_A = row['player_name_A']
        player_name_B = row['player_name_B']
        team_name_A = row['team_name_A']
        team_name_B = row['team_name_B']
        birth_date_A = row['birth_date_A']
        birth_date_B = row['birth_date_B']
        weight_A = row['weight_A']
        height_A = row['height_A']
        weight_B = row['weight_B']
        height_B = row['height_B']
        specialized_technique_A = row['specialized_technique_A']
        specialized_technique_B = row['specialized_technique_B']
        first_learning_year_A = row['first_learning_year_A']
        first_learning_year_B = row['first_learning_year_B']
        pro_debut_year_A = row['pro_debut_year_A']
        pro_debut_year_B = row['pro_debut_year_B']
        text = row['text_relay']

        article = f'{play_date}일 열린 {contest_name} {play_level}에서 {player_name_A}({team_name_A}) 선수와 {player_name_B}({team_name_B}) 선수가 맞붙었다.\n청샅바인 {player_name_A} 선수는 {birth_date_A} 출생으로 신장은 {height_A} 이며 체중은 {weight_A}이다.\n{pro_debut_year_A} 씨름대회에 데뷔하였고 {specialized_technique_A} 기술이 주특기이다.\n홍샅바인 {player_name_B} 선수는 {birth_date_B} 출생으로 신장은 {height_B}이며 체중은 {weight_B}이다.\n{pro_debut_year_B} 씨름대회에 데뷔하였고 {specialized_technique_B} 기술이 주특기이다.'

        if '기권' in text_relay:
            print(f'<{player_name_A}와 {player_name_B}, {contest_name} {play_level}에서 무승부 기록>\n' + article + ' 양 선수의 기권으로 인해 해당 경기는 승패를 결정짓지 못하였다.')
            continue
            
        if '청' in winner:
            title = f'<{player_name_A}, {contest_name} {play_level}에서 {player_name_B} 꺾고 다음 라운드 진출>\n'


        elif '홍' in winner:
            title = f'<{player_name_B}, {contest_name} {play_level}에서 {player_name_A} 꺾고 다음 라운드 진출>\n'

        
        # 텍스트 데이터를 줄 단위로 분할
        lines = text.strip().split('\n')

        # DataFrame을 만들기 위한 빈 리스트 초기화
        data = []
        round_number = 0
        time = 0

        # 주어진 텍스트를 분석하여 데이터 추출
        for line in lines:
            if line.endswith("라운드 경기시작"):
                round_number += 1
                time = 0
            elif line.endswith("초"):
                time = 60 - int(line[:-1])
            else:
                data.append([round_number, time, line])

        def change_word(text):
            word_list = str(text).split(' ')
            word_list.insert(2, '선수가')
            sentence = ' '.join(word_list)
            return sentence
        
        # DataFrame 생성
        new_df = pd.DataFrame(data, columns=['round', 'second', 'detail'])
        new_df['detail'] = new_df['detail'].apply(lambda x:str(x).replace('홍 ', '홍샅바 ') if str(x).startswith('홍 ') else (str(x).replace('청 ', '청샅바 ') if str(x).startswith('청 ') else (str(x))))
        new_df = new_df[(new_df['detail'] != '청샅바 {}'.format(player_name_A)) & (new_df['detail'] != '홍샅바 {}'.format(player_name_B))]
        new_df = new_df[new_df['detail'].str.contains('공격시도|기술방어|승리기술')]
        new_df.reset_index(drop=True, inplace=True)
        new_df['detail'] = new_df['detail'].apply(lambda x: change_word(x))

        
        max_round = new_df['round'].max()
        
        # 1라운드에서 경기가 종료된 경우
        if max_round == 1:
            round1 = new_df[new_df['round'] == 1][['second' ,'detail']].values
            sentence1 = ''
            for second, detail in round1:
                sentence1 += ('{}초가 경과되었을 때 {}. '.format(second, detail))

            modified_sentence1 = merge_sentences(sentence1)
            modified_sentence1 = modified_sentence1.replace('  ', ' ')
        

            print(title + article + ' 1라운드 시작 직후 ' + modified_sentence1 + f'최종 스코어 {match_result[0][0]}:{match_result[0][3]}으로 {winner[-3:]} 선수가 승리하며 {play_level}에서 다음 라운드로 진출하였다.')
        
        # 2라운드에서 경기가 종료된 경우
        elif max_round == 2:
            round1 = new_df[new_df['round'] == 1][['second' ,'detail']].values
            sentence1 = ''
            for second, detail in round1:
                sentence1 += ('{}초가 경과되었을 때 {}. '.format(second, detail))

            modified_sentence1 = merge_sentences(sentence1)
            modified_sentence1 = modified_sentence1.replace('  ', ' ')

            round2 = new_df[new_df['round'] == 2][['second' ,'detail']].values
            sentence2 = ''
            for second, detail in round2:
                sentence2 += ('{}초가 경과되었을 때 {}. '.format(second, detail))

            modified_sentence2 = merge_sentences(sentence2)
            modified_sentence2 = modified_sentence2.replace('  ', ' ')

            print(title + article + ' 1라운드 시작 직후 ' + modified_sentence1 + '\n 2라운드 시작 직후 ' + modified_sentence2 + f'최종 스코어 {match_result[0][0]}:{match_result[0][3]}으로 {winner[-3:]} 선수가 승리하며 {play_level}에서 다음 라운드로 진출하였다.')
        
        # 3라운드에서 경기가 종료된 경우
        elif max_round == 3:
            round1 = new_df[new_df['round'] == 1][['second' ,'detail']].values
            sentence1 = ''
            for second, detail in round1:
                sentence1 += ('{}초가 경과되었을 때 {}. '.format(second, detail))

            modified_sentence1 = merge_sentences(sentence1)
            modified_sentence1 = modified_sentence1.replace('  ', ' ')

            round2 = new_df[new_df['round'] == 2][['second' ,'detail']].values
            sentence2 = ''
            for second, detail in round2:
                sentence2 += ('{}초가 경과되었을 때 {}. '.format(second, detail))

            modified_sentence2 = merge_sentences(sentence2)
            modified_sentence2 = modified_sentence2.replace('  ', ' ')

            round3 = new_df[new_df['round'] == 3][['second' ,'detail']].values
            sentence3 = ''
            for second, detail in round3:
                sentence3 += ('{}초가 경과되었을 때 {}. '.format(second, detail))

            modified_sentence3 = merge_sentences(sentence3)
            modified_sentence3 = modified_sentence3.replace('  ', ' ')

            print(title + article + ' 1라운드 시작 직후 ' + modified_sentence1 + '\n 2라운드 시작 직후 ' + modified_sentence2 + '\n 3라운드 시작 직후 ' + modified_sentence3 + f'최종 스코어 {match_result[0][0]}:{match_result[0][3]}으로 {winner[-3:]} 선수가 승리하며 {play_level}에서 다음 라운드로 진출하였다.')
        
        else:
            print(title + article + ' ' + f'경기는 {winner[-3:]} 선수가 승리하며 {play_level}에서 다음 라운드로 진출하였다.')
        

In [38]:
text_relay_modifier(df_) # 비디오 판독 결과를 통해 라운드 승패 결판이 난 경우는 아직 적용하지 못했음. 전반적으로 문장이 말은 되나 초단위로 경기진행이 되다보니 기사라기 보다 나열되어 있는 말뭉치 느낌이 든다. 날릴 정보는 날리는게 좋을 듯


1 row processing...
<정재욱, 보은장사씨름대회 1조 32강에서 황성희 꺾고 다음 라운드 진출>
5.20(토)일 열린 보은장사씨름대회 1조 32강에서 정재욱(구미시청씨름단) 선수와 황성희(울주군청해뜨미씨름단) 선수가 맞붙었다.
청샅바인 정재욱 선수는 1987 년 06 월 18 일 출생으로 신장은 175cm 이며 체중은 82kg이다.
2010년 씨름대회에 데뷔하였고 안다리 기술이 주특기이다.
홍샅바인 황성희 선수는 1994 년 02 월 21 일 출생으로 신장은 176cm이며 체중은 95kg이다.
2017년 씨름대회에 데뷔하였고 들배지기 기술이 주특기이다. 경기는 정재욱 선수가 승리하며 1조 32강에서 다음 라운드로 진출하였다.

2 row processing...
<김기선, 보은장사씨름대회 1조 32강에서 홍성용 꺾고 다음 라운드 진출>
5.20(토)일 열린 보은장사씨름대회 1조 32강에서 홍성용(양평군청씨름단) 선수와 김기선(연수구청씨름단) 선수가 맞붙었다.
청샅바인 홍성용 선수는 1992 년 04 월 29 일 출생으로 신장은 178cm 이며 체중은 90kg이다.
2015년 씨름대회에 데뷔하였고 오금당기기 기술이 주특기이다.
홍샅바인 김기선 선수는 1992 년 01 월 12 일 출생으로 신장은 172cm이며 체중은 92kg이다.
2015년 씨름대회에 데뷔하였고 오금당기기 기술이 주특기이다. 경기는 김기선 선수가 승리하며 1조 32강에서 다음 라운드로 진출하였다.

3 row processing...
<김민정, 보은장사씨름대회 1조 32강에서 이설빈 꺾고 다음 라운드 진출>
5.20(토)일 열린 보은장사씨름대회 1조 32강에서 이설빈(창원특례시청씨름단) 선수와 김민정(영월군청씨름단) 선수가 맞붙었다.
청샅바인 이설빈 선수는 1996 년 03 월 29 일 출생으로 신장은 178cm 이며 체중은 85kg이다.
2020년 씨름대회에 데뷔하였고 왼배지기 기술이 주특기이다.
홍샅바인 김민정 선수는 1988 년 04 월 09 일 출생으로 신장은 187cm이

##### 문장개선을 위한 여러가지 테스트용 코드(이 밑에 코드는 무시하면 됨)

In [65]:
import pandas as pd

# 주어진 텍스트 데이터
text = """
1라운드 경기시작
58초
홍 김민정
홍 김민정 허리기술(들배지기) 공격시도
58초
청 이설빈
청 이설빈 기술방어
57초
홍 김민정
홍 김민정 다리기술(호미걸이) 공격시도
57초
청 이설빈
청 이설빈 기술방어
55초
청 이설빈
청 이설빈 다리기술(안다리걸기) 승리기술
55초
청 이설빈
비디오판독(감독) 판정유지
1라운드 경기종료
청 승
2라운드 경기시작
59초
홍 김민정
홍 김민정 허리기술(잡채기) 승리기술
2라운드 경기종료
홍 승
3라운드 경기시작
59초
홍 김민정
홍 김민정 허리기술(들배지기) 공격시도
59초
청 이설빈
청 이설빈 기술방어
59초
홍 김민정
홍 김민정 다리기술(호미걸이) 공격시도
59초
청 이설빈
청 이설빈 기술방어
57초
홍 김민정
홍 김민정 혼합기술(밀어치기) 승리기술
3라운드 경기종료
홍 승
경기종료 
홍 승
"""

# 텍스트 데이터를 줄 단위로 분할
lines = text.strip().split('\n')

# DataFrame을 만들기 위한 빈 리스트 초기화
data = []
round_number = 0
time = 0

# 주어진 텍스트를 분석하여 데이터 추출
for line in lines:
    if line.endswith("라운드 경기시작"):
        round_number += 1
        time = 0
    elif line.endswith("초"):
        time = 60 - int(line[:-1])
    else:
        data.append([round_number, time, line])

def change_word(text):
    word_list = str(text).split(' ')
    word_list.insert(2, '선수가')
    sentence = ' '.join(word_list)
    return sentence

# DataFrame 생성
df = pd.DataFrame(data, columns=["라운드", "초", "내용"])
df['내용'] = df['내용'].apply(lambda x:str(x).replace('홍 ', '홍샅바 ') if str(x).startswith('홍 ') else (str(x).replace('청 ', '청샅바 ') if str(x).startswith('청 ') else (str(x))))
df = df[df['내용'].str.contains('공격시도|기술방어|승리기술')]
df.reset_index(drop=True, inplace=True)
df['내용'] = df['내용'].apply(lambda x: change_word(x))
print(df)

round1 = df[df['라운드'] == 1][['초' ,'내용']].values
sentence1 = ''
for second, detail in round1:
    sentence1 += ('{}초가 경과되었을 때 {}. '.format(second, detail))

sentence1 = '1라운드 시작 직후 ' + sentence1

round2 = df[df['라운드'] == 2][['초' ,'내용']].values
sentence2 = ''
for second, detail in round1:
    sentence2 += ('{}초가 경과되었을 때 {}. '.format(second, detail))

sentence2 = '2라운드 시작 직후 ' + sentence2

round3 = df[df['라운드'] == 3][['초' ,'내용']].values
sentence3 = ''
for second, detail in round3:
    sentence3 += ('{}초가 경과되었을 때 {}. '.format(second, detail))

sentence3 = '3라운드 시작 직후 ' + sentence3

print(sentence1 + sentence2 + sentence3)
print(merge_sentences(sentence1 + sentence2 + sentence3))


# '1라운드 시작 초가 경과되었을 때 홍샅바 김민정 선수가'


    라운드  초                            내용
0     1  2   홍샅바 김민정 선수가 허리기술(들배지기) 공격시도
1     1  2              청샅바 이설빈 선수가 기술방어
2     1  3   홍샅바 김민정 선수가 다리기술(호미걸이) 공격시도
3     1  3              청샅바 이설빈 선수가 기술방어
4     1  5  청샅바 이설빈 선수가 다리기술(안다리걸기) 승리기술
5     2  1    홍샅바 김민정 선수가 허리기술(잡채기) 승리기술
6     3  1   홍샅바 김민정 선수가 허리기술(들배지기) 공격시도
7     3  1              청샅바 이설빈 선수가 기술방어
8     3  1   홍샅바 김민정 선수가 다리기술(호미걸이) 공격시도
9     3  1              청샅바 이설빈 선수가 기술방어
10    3  3   홍샅바 김민정 선수가 혼합기술(밀어치기) 승리기술
1라운드 시작 직후 2초가 경과되었을 때 홍샅바 김민정 선수가 허리기술(들배지기) 공격시도. 2초가 경과되었을 때 청샅바 이설빈 선수가 기술방어. 3초가 경과되었을 때 홍샅바 김민정 선수가 다리기술(호미걸이) 공격시도. 3초가 경과되었을 때 청샅바 이설빈 선수가 기술방어. 5초가 경과되었을 때 청샅바 이설빈 선수가 다리기술(안다리걸기) 승리기술. 2라운드 시작 직후 2초가 경과되었을 때 홍샅바 김민정 선수가 허리기술(들배지기) 공격시도. 2초가 경과되었을 때 청샅바 이설빈 선수가 기술방어. 3초가 경과되었을 때 홍샅바 김민정 선수가 다리기술(호미걸이) 공격시도. 3초가 경과되었을 때 청샅바 이설빈 선수가 기술방어. 5초가 경과되었을 때 청샅바 이설빈 선수가 다리기술(안다리걸기) 승리기술. 3라운드 시작 직후 1초가 경과되었을 때 홍샅바 김민정 선수가 허리기술(들배지기) 공격시도. 1초가 경과되었을 때 청샅바 이설빈 선수가 기술방어. 1초가 경과되었을 때 홍샅바 김민정 선수가 다리기술(호미걸이) 공